<a href="https://colab.research.google.com/github/KendallScott/QTW/blob/main/Case%20Study%203/Case_Study_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [70]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [92]:
pip install python-magic

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [95]:
import quopri
import codecs
import magic
import email

i=0

contents = []
types = []
labels = []

for root, dirs, files in os.walk("/content/drive/MyDrive/SpamAssassinMessages/"):
    for name in files:
        with open(os.path.join(root,name),'r') as f:
            try:
                x = email.message_from_file(f)
                i+=1

            except UnicodeDecodeError:  
                blob = open(file_list[i], 'rb').read()
                m = magic.Magic(mime_encoding=True)
                encoding = m.from_buffer(blob)
                
                contents.append(encoding)
                print(file_list[i])
                i+=1
                
            if "multipart" in x.get_content_type():
                # do something
                contents.append("")

            if "text/plain" in x.get_content_type():  #boy I hope all text messages are the same!
                contents.append(x.get_payload().replace("\n"," "))
            elif "html" in x.get_content_type():
                # do something else

                contents.append("")
            types.append(x.get_content_type())
            if "ham" in root:
                labels.append(0)
            elif "spam" in root:
                labels.append(1)
            else:
                print("YOU HAVE A BIG PROBLEM--LABEL NOT FOUND")
                
        

/content/drive//MyDrive//SpamAssassinMessages//spam_2/00471.df77fa930951f79466c195052ff56816
/content/drive//MyDrive//SpamAssassinMessages//spam_2/00580.c3b23134b4767f5e796d0df997fede33
/content/drive//MyDrive//SpamAssassinMessages//spam_2/00464.d2f719c667d192af860572b1c858cc11
/content/drive//MyDrive//SpamAssassinMessages//spam_2/00609.4dfe7912017772587dc62fecc3cf6553
/content/drive//MyDrive//SpamAssassinMessages//spam_2/00468.fb2222cc49228bd3dac5481e670f208f
/content/drive//MyDrive//SpamAssassinMessages//spam_2/00472.3c51cf86307bc98c54d856887a81e9ac
/content/drive//MyDrive//SpamAssassinMessages//spam_2/00556.098b57f5108ba34d21825f176e492786
/content/drive//MyDrive//SpamAssassinMessages//spam_2/00645.dd7d8ec1eb687c5966c516b720fcc3d5
/content/drive//MyDrive//SpamAssassinMessages//spam_2/00750.dfc392478300e11189d61d29bed9cecc
/content/drive//MyDrive//SpamAssassinMessages//spam_2/00706.5116018237368c3633823b2d24f8ac86
/content/drive//MyDrive//SpamAssassinMessages//spam_2/00743.7889a6b188

In [97]:
len(contents)

10106

In [98]:
len(types)

9353

In [99]:
len(labels)

9353

In [100]:
len(contents)

10106

In [102]:
import pandas as pd 

data = pd.DataFrame({"Text":contents,"Label":labels})

ValueError: ignored

In [ ]:
data[data['type'].str.contains("html")]

In [ ]:
data

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
vectorizer = CountVectorizer()

In [ ]:
out = vectorizer.fit_transform(data['Text'])

In [ ]:
vectorizer.vocabulary_

In [ ]:
data.iloc[0,0]

In [ ]:
<unk>
            

In [ ]:
reverse={}
for word,value in vectorizer.vocabulary_.items():
    reverse[value]=word

In [ ]:
reverse[64792]

In [ ]:
for i in range(out[0].shape[1]):
    if out[0,i]>0:
        print(reverse[i], out[0,i])

In [ ]:
out[0,0]

In [ ]:
vectorizer.vocabulary_['unk']

In [ ]:
<unk> = 102
<start> = 101
<end>= 3
<pad> = 100
<cls>

In [ ]:
"<pad> <pad> <start> The <unk> car drove away <end>"

In [ ]:
from sklearn.datasets import fetch_20newsgroups
data = fetch_20newsgroups(subset='train')
test = fetch_20newsgroups(subset='test')

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
word_counts = CountVectorizer()
v_data = word_counts.fit_transform(data['data'])
v_test = word_counts.transform(test['data'])

In [ ]:
v_data

In [ ]:
from sklearn.naive_bayes import ComplementNB, BernoulliNB, GaussianNB, CategoricalNB, MultinomialNB
from sklearn.metrics import accuracy_score, classification_report, ConfusionMatrixDisplay
comp = ComplementNB()
bern = BernoulliNB()
gaus = GaussianNB()
cat = CategoricalNB()
multi = MultinomialNB()
methods = {"Complement":comp,"Bernoulli":bern,"Gausian":gaus,"Multi":multi,"Cat":cat}
for i in methods.keys():
    try:
        methods[i].fit(v_data,data['target'])
        p = methods[i].predict(v_test)
        print(accuracy_score(test['target'],p), f"method={i}")
        print(classification_report(test['target'],p))
    except:
        print(f"{i} failed")


In [ ]:
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = (20,20)
ConfusionMatrixDisplay.from_predictions(test['target'],p,cmap='Blues')

In [ ]:
reverse={}
for word,value in word_counts.vocabulary_.items():
    reverse[value]=word